In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from src.reward_model import RewardModel, PreferenceDataset, preference_loss
from torch.utils.data import DataLoader
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Example synthetic dataset
data = [
    {"prompt": "Why did the website crash?",
     "winning": "Too many users logging in caused a memory overflow.",
     "losing": "Aliens disabled the server."},
    {"prompt": "Why did the train stop?",
     "winning": "Signal failure on the main line.",
     "losing": "Because the sun forgot to rise."}
]

dataset = PreferenceDataset(tokenizer, data)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RewardModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training step
for epoch in range(3):
    for batch in dataloader:
        optimizer.zero_grad()

        winning_scores = model(batch["winning_input_ids"].to(device),
                              batch["winning_mask"].to(device))

        losing_scores = model(batch["losing_input_ids"].to(device),
                                batch["losing_mask"].to(device))

        loss = preference_loss(winning_scores, losing_scores)

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
d:\Projects\RLHF\rlhf\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\binit\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://do

Epoch 1 | Loss: 0.7265
Epoch 2 | Loss: 0.6789
Epoch 3 | Loss: 0.6368


In [ ]:
output = model()
output

tensor([-0.2411, -0.1654], grad_fn=<SqueezeBackward1>)

In [2]:
from datasets import load_dataset

ds = load_dataset("nvidia/HelpSteer2")

d:\Projects\RLHF\rlhf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Projects\RLHF\rlhf\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\binit\.cache\huggingface\hub\datasets--nvidia--HelpSteer2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: 

In [8]:
ds["train"][90]

{'prompt': "I would like you to audit content for me based on a URL I specify below. I would like you to audit content as if you were a Google Quality Rater following the rules set out by Google (which you can see here (https://developers.google.com/search/blog/2022/08/helpful-content-update)in respect of August 2022 helpful content update (experience, expertise, authority and trust) - I would also like you to consider YMYL (your money your life where applicable) and Google medic factors also depending on the content type and nature. I would like you to provide a content quality rating based on a scale of 1 to 10 where 10 is best and 0 is worst. You should take into consideration - how well the content is written, how well it aligns with Google's August 2022 helpful content update guidelines for human quality raters, how well structured the content is, if it makes it clear what is on offer, is it gramatically correct and well written and does it fit the end users intent when comparing 